# ML Model Horse Racing Predictions
#### Create an ML model that can predict whether a horse has a good chance of winning a race, based on the horse's features

## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
cleaned_data_df = pd.read_csv("Data_Racing/International_Data/cleaned_data_df.csv")
cleaned_data_df.head()

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


2023-07-19 18:31:47.548149: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2, in other operations, rebuild TensorFlow with the appropriate compiler flags.


,Unnamed: 0,ages,metric,ncond,age,saddle,decimalPrice,isFav,RPR,TR,OR,father,weight,res_win
0,1,4yo+,3218.0,10,6.0,7.0,0.444444,1,96.0,89.0,110.0,Getaway,65,0.0
1,26,5yo+,3218.0,5,8.0,1.0,0.266667,1,115.0,33.0,109.0,Beneficial,73,0.0
2,27,5yo+,3218.0,5,9.0,2.0,0.125000,0,108.0,29.0,103.0,Beneficial,68,0.0
3,43,4yo+,4827.0,6,7.0,13.0,0.076923,0,95.0,39.0,83.0,Kalanisi,63,1.0
4,44,4yo+,4827.0,6,8.0,2.0,0.076923,0,119.0,64.0,108.0,Ask,71,0.0


In [2]:
cleaned_data_df = cleaned_data_df.drop(columns = "Unnamed: 0")

In [3]:
# Determine the number of unique values in each column.
cleaned_data_df.nunique()

ages             16
metric           52
ncond            13
age              14
saddle           29
decimalPrice     79
isFav             2
RPR             165
TR              157
OR              149
father          327
weight           32
res_win           2
dtype: int64

## Select equal res_win values

In [4]:
# Look at value counts for binning
res_win_counts = cleaned_data_df["res_win"].value_counts()
class_count = res_win_counts.min()
class_count

3169

In [5]:
res_win_equal = pd.concat([cleaned_data_df[cleaned_data_df.res_win==0].sample(n=class_count), 
                           cleaned_data_df[cleaned_data_df.res_win==1].sample(n=class_count)])
res_win_equal = res_win_equal.sort_index()
res_win_equal

,ages,metric,ncond,age,saddle,decimalPrice,isFav,RPR,TR,OR,father,weight,res_win
1,5yo+,3218.0,5,8.0,1.0,0.266667,1,115.0,33.0,109.0,Beneficial,73,0.0
3,4yo+,4827.0,6,7.0,13.0,0.076923,0,95.0,39.0,83.0,Kalanisi,63,1.0
4,4yo+,4827.0,6,8.0,2.0,0.076923,0,119.0,64.0,108.0,Ask,71,0.0
10,4yo+,1005.0,0,5.0,4.0,0.047619,0,91.0,69.0,83.0,Showcasing,58,1.0
12,4yo+,1005.0,0,9.0,1.0,0.181818,0,83.0,60.0,86.0,Exceed And Excel,60,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
24897,3yo+,1609.0,0,6.0,9.0,0.181818,0,90.0,41.0,82.0,Rio De La Plata,56,1.0
24905,4yo+,3821.0,5,6.0,2.0,0.222222,0,134.0,40.0,135.0,Jeremy,69,1.0
24913,4yo+,4022.0,5,7.0,5.0,0.153846,0,144.0,82.0,129.0,Poliglote,70,1.0
24919,4yo+,4725.5,9,8.0,1.0,0.076923,0,141.0,3.0,139.0,Shantou,75,1.0


## Convert and Standardize Data

In [26]:
# Convert categorical data to numeric with `pd.get_dummies`
converted_data = pd.get_dummies(res_win_equal)
converted_data.head()

,metric,ncond,age,saddle,decimalPrice,isFav,RPR,TR,OR,weight,...,father_War Front,father_Well Chosen,father_Westerner,father_Whipper,father_Winged Love,father_Wootton Bassett,father_Yeats,father_Youmzain,father_Zebedee,father_Zoffany
1,3218.0,5,8.0,1.0,0.266667,1,115.0,33.0,109.0,73,...,0,0,0,0,0,0,0,0,0,0
3,4827.0,6,7.0,13.0,0.076923,0,95.0,39.0,83.0,63,...,0,0,0,0,0,0,0,0,0,0
4,4827.0,6,8.0,2.0,0.076923,0,119.0,64.0,108.0,71,...,0,0,0,0,0,0,0,0,0,0
10,1005.0,0,5.0,4.0,0.047619,0,91.0,69.0,83.0,58,...,0,0,0,0,0,0,0,0,0,0
12,1005.0,0,9.0,1.0,0.181818,0,83.0,60.0,86.0,60,...,0,0,0,0,0,0,0,0,0,0


In [27]:
# Split our preprocessed data into our features and target arrays
y = converted_data['res_win']
X = converted_data.drop(columns=['res_win'])

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [28]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [29]:
input_features = len(X_train_scaled[0])
input_features

336

## Compile, Train and Evaluate the Model

In [30]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_features = len(X_train_scaled[0])
neurons_hidden_layer1 = 356
neurons_hidden_layer2 = 248
neurons_hidden_layer3 = 176

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units = neurons_hidden_layer1, activation="relu", input_dim = input_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units = neurons_hidden_layer2, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units = neurons_hidden_layer3, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 356)               119972    
                                                                 
 dense_9 (Dense)             (None, 248)               88536     
                                                                 
 dense_10 (Dense)            (None, 176)               43824     
                                                                 
 dense_11 (Dense)            (None, 1)                 177       
                                                                 
Total params: 252,509
Trainable params: 252,509
Non-trainable params: 0
_________________________________________________________________


In [31]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [32]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs = 1000)

Epoch 1/1000
149/149 [==============================] - 1s 1ms/step - loss: 0.6343 - accuracy: 0.6472
Epoch 2/1000
149/149 [==============================] - 0s 1ms/step - loss: 0.5273 - accuracy: 0.7288
Epoch 3/1000
149/149 [==============================] - 0s 1ms/step - loss: 0.4770 - accuracy: 0.7593
Epoch 4/1000
149/149 [==============================] - 0s 1ms/step - loss: 0.4401 - accuracy: 0.7789
Epoch 5/1000
149/149 [==============================] - 0s 1ms/step - loss: 0.4037 - accuracy: 0.7980
Epoch 6/1000
149/149 [==============================] - 0s 1ms/step - loss: 0.3800 - accuracy: 0.8142
Epoch 7/1000
149/149 [==============================] - 0s 1ms/step - loss: 0.3521 - accuracy: 0.8285
Epoch 8/1000
149/149 [==============================] - 0s 1ms/step - loss: 0.3338 - accuracy: 0.8468
Epoch 9/1000
149/149 [==============================] - 0s 1ms/step - loss: 0.2982 - accuracy: 0.8548
Epoch 10/1000
149/149 [==============================] - 0s 1ms/step - loss: 0.285

149/149 [==============================] - 0s 1ms/step - loss: 0.0109 - accuracy: 0.9971
Epoch 81/1000
149/149 [==============================] - 0s 1ms/step - loss: 0.0117 - accuracy: 0.9966
Epoch 82/1000
149/149 [==============================] - 0s 2ms/step - loss: 0.0059 - accuracy: 0.9975
Epoch 83/1000
149/149 [==============================] - 0s 2ms/step - loss: 0.0051 - accuracy: 0.9979
Epoch 84/1000
149/149 [==============================] - 0s 2ms/step - loss: 0.0091 - accuracy: 0.9975
Epoch 85/1000
149/149 [==============================] - 0s 1ms/step - loss: 0.0088 - accuracy: 0.9977
Epoch 86/1000
149/149 [==============================] - 0s 1ms/step - loss: 0.0274 - accuracy: 0.9905
Epoch 87/1000
149/149 [==============================] - 0s 1ms/step - loss: 0.0311 - accuracy: 0.9891
Epoch 88/1000
149/149 [==============================] - 0s 1ms/step - loss: 0.0297 - accuracy: 0.9920
Epoch 89/1000
149/149 [==============================] - 0s 1ms/step - loss: 0.0541 - a

149/149 [==============================] - 0s 1ms/step - loss: 0.0019 - accuracy: 0.9987
Epoch 160/1000
149/149 [==============================] - 0s 1ms/step - loss: 0.0012 - accuracy: 0.9994
Epoch 161/1000
149/149 [==============================] - 0s 1ms/step - loss: 0.0010 - accuracy: 0.9994
Epoch 162/1000
149/149 [==============================] - 0s 1ms/step - loss: 0.0011 - accuracy: 0.9996
Epoch 163/1000
149/149 [==============================] - 0s 1ms/step - loss: 0.0013 - accuracy: 0.9992
Epoch 164/1000
149/149 [==============================] - 0s 1ms/step - loss: 9.3980e-04 - accuracy: 0.9996
Epoch 165/1000
149/149 [==============================] - 0s 1ms/step - loss: 9.5467e-04 - accuracy: 0.9994
Epoch 166/1000
149/149 [==============================] - 0s 1ms/step - loss: 8.5423e-04 - accuracy: 0.9996
Epoch 167/1000
149/149 [==============================] - 0s 1ms/step - loss: 8.9608e-04 - accuracy: 0.9996
Epoch 168/1000
149/149 [==============================] - 0s 1m

149/149 [==============================] - 0s 1ms/step - loss: 0.0015 - accuracy: 0.9992
Epoch 237/1000
149/149 [==============================] - 0s 1ms/step - loss: 0.0012 - accuracy: 0.9994
Epoch 238/1000
149/149 [==============================] - 0s 1ms/step - loss: 0.0011 - accuracy: 0.9996
Epoch 239/1000
149/149 [==============================] - 0s 1ms/step - loss: 8.8758e-04 - accuracy: 0.9998
Epoch 240/1000
149/149 [==============================] - 0s 1ms/step - loss: 7.7543e-04 - accuracy: 0.9996
Epoch 241/1000
149/149 [==============================] - 0s 1ms/step - loss: 6.2910e-04 - accuracy: 0.9996
Epoch 242/1000
149/149 [==============================] - 0s 1ms/step - loss: 6.3250e-04 - accuracy: 0.9996
Epoch 243/1000
149/149 [==============================] - 0s 1ms/step - loss: 8.3489e-04 - accuracy: 0.9994
Epoch 244/1000
149/149 [==============================] - 0s 1ms/step - loss: 6.6187e-04 - accuracy: 0.9998
Epoch 245/1000
149/149 [==============================]

149/149 [==============================] - 0s 1ms/step - loss: 0.1051 - accuracy: 0.9701
Epoch 313/1000
149/149 [==============================] - 0s 1ms/step - loss: 0.0567 - accuracy: 0.9806
Epoch 314/1000
149/149 [==============================] - 0s 1ms/step - loss: 0.0251 - accuracy: 0.9926
Epoch 315/1000
149/149 [==============================] - 0s 1ms/step - loss: 0.0175 - accuracy: 0.9958
Epoch 316/1000
149/149 [==============================] - 0s 1ms/step - loss: 0.0110 - accuracy: 0.9975
Epoch 317/1000
149/149 [==============================] - 0s 1ms/step - loss: 0.0136 - accuracy: 0.9956
Epoch 318/1000
149/149 [==============================] - 0s 1ms/step - loss: 0.0049 - accuracy: 0.9985
Epoch 319/1000
149/149 [==============================] - 0s 1ms/step - loss: 0.0021 - accuracy: 0.9989
Epoch 320/1000
149/149 [==============================] - 0s 1ms/step - loss: 0.0066 - accuracy: 0.9983
Epoch 321/1000
149/149 [==============================] - 0s 1ms/step - loss: 0

...

Epoch 693/1000
149/149 [==============================] - 0s 1ms/step - loss: 2.1288e-10 - accuracy: 1.0000
Epoch 694/1000
149/149 [==============================] - 0s 1ms/step - loss: 1.9443e-10 - accuracy: 1.0000
Epoch 695/1000
149/149 [==============================] - 0s 1ms/step - loss: 1.9767e-10 - accuracy: 1.0000
Epoch 696/1000
149/149 [==============================] - 0s 1ms/step - loss: 1.9015e-10 - accuracy: 1.0000
Epoch 697/1000
149/149 [==============================] - 0s 1ms/step - loss: 1.8099e-10 - accuracy: 1.0000
Epoch 698/1000
149/149 [==============================] - 0s 1ms/step - loss: 1.8525e-10 - accuracy: 1.0000
Epoch 699/1000
149/149 [==============================] - 0s 1ms/step - loss: 1.7204e-10 - accuracy: 1.0000
Epoch 700/1000
149/149 [==============================] - 0s 1ms/step - loss: 1.7210e-10 - accuracy: 1.0000
Epoch 701/1000
149/149 [==============================] - 0s 1ms/step - loss: 1.5869e-10 - accuracy: 1.0000
Epoch 702/1000
149/149 [====

149/149 [==============================] - 0s 2ms/step - loss: 8.8454e-11 - accuracy: 1.0000
Epoch 769/1000
149/149 [==============================] - 0s 1ms/step - loss: 8.4211e-11 - accuracy: 1.0000
Epoch 770/1000
149/149 [==============================] - 0s 1ms/step - loss: 8.6675e-11 - accuracy: 1.0000
Epoch 771/1000
149/149 [==============================] - 0s 1ms/step - loss: 8.1429e-11 - accuracy: 1.0000
Epoch 772/1000
149/149 [==============================] - 0s 1ms/step - loss: 8.4130e-11 - accuracy: 1.0000
Epoch 773/1000
149/149 [==============================] - 0s 1ms/step - loss: 8.6914e-11 - accuracy: 1.0000
Epoch 774/1000
149/149 [==============================] - 0s 1ms/step - loss: 8.7902e-11 - accuracy: 1.0000
Epoch 775/1000
149/149 [==============================] - 0s 1ms/step - loss: 8.7651e-11 - accuracy: 1.0000
Epoch 776/1000
149/149 [==============================] - 0s 1ms/step - loss: 8.3084e-11 - accuracy: 1.0000
Epoch 777/1000
149/149 [===================

Epoch 844/1000
149/149 [==============================] - 0s 1ms/step - loss: 7.8489e-11 - accuracy: 1.0000
Epoch 845/1000
149/149 [==============================] - 0s 1ms/step - loss: 8.0372e-11 - accuracy: 1.0000
Epoch 846/1000
149/149 [==============================] - 0s 1ms/step - loss: 7.7695e-11 - accuracy: 1.0000
Epoch 847/1000
149/149 [==============================] - 0s 1ms/step - loss: 7.0992e-11 - accuracy: 1.0000
Epoch 848/1000
149/149 [==============================] - 0s 1ms/step - loss: 6.4463e-11 - accuracy: 1.0000
Epoch 849/1000
149/149 [==============================] - 0s 1ms/step - loss: 6.4993e-11 - accuracy: 1.0000
Epoch 850/1000
149/149 [==============================] - 0s 1ms/step - loss: 6.5903e-11 - accuracy: 1.0000
Epoch 851/1000
149/149 [==============================] - 0s 1ms/step - loss: 6.7158e-11 - accuracy: 1.0000
Epoch 852/1000
149/149 [==============================] - 0s 1ms/step - loss: 6.9402e-11 - accuracy: 1.0000
Epoch 853/1000
149/149 [====

149/149 [==============================] - 0s 1ms/step - loss: 6.4122e-11 - accuracy: 1.0000
Epoch 920/1000
149/149 [==============================] - 0s 1ms/step - loss: 6.4711e-11 - accuracy: 1.0000
Epoch 921/1000
149/149 [==============================] - 0s 1ms/step - loss: 6.2113e-11 - accuracy: 1.0000
Epoch 922/1000
149/149 [==============================] - 0s 1ms/step - loss: 5.5477e-11 - accuracy: 1.0000
Epoch 923/1000
149/149 [==============================] - 0s 1ms/step - loss: 5.6844e-11 - accuracy: 1.0000
Epoch 924/1000
149/149 [==============================] - 0s 1ms/step - loss: 5.7481e-11 - accuracy: 1.0000
Epoch 925/1000
149/149 [==============================] - 0s 1ms/step - loss: 5.8374e-11 - accuracy: 1.0000
Epoch 926/1000
149/149 [==============================] - 0s 1ms/step - loss: 5.9014e-11 - accuracy: 1.0000
Epoch 927/1000
149/149 [==============================] - 0s 1ms/step - loss: 5.9786e-11 - accuracy: 1.0000
Epoch 928/1000
149/149 [===================

Epoch 995/1000
149/149 [==============================] - 0s 1ms/step - loss: 6.2596e-11 - accuracy: 1.0000
Epoch 996/1000
149/149 [==============================] - 0s 1ms/step - loss: 6.3449e-11 - accuracy: 1.0000
Epoch 997/1000
149/149 [==============================] - 0s 1ms/step - loss: 6.3787e-11 - accuracy: 1.0000
Epoch 998/1000
149/149 [==============================] - 0s 1ms/step - loss: 6.4545e-11 - accuracy: 1.0000
Epoch 999/1000
149/149 [==============================] - 0s 1ms/step - loss: 6.4846e-11 - accuracy: 1.0000
Epoch 1000/1000
149/149 [==============================] - 0s 1ms/step - loss: 6.5328e-11 - accuracy: 1.0000


In [33]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

50/50 - 0s - loss: 15.9221 - accuracy: 0.7798 - 270ms/epoch - 5ms/step
Loss: 15.922137260437012, Accuracy: 0.7798107266426086


In [34]:
(y_test == 0).mean()

0.4965299684542587

In [35]:
# Make predictions on the test data
y_pred = nn.predict(X_test_scaled)
y_pred_binary = (y_pred > 0.5).astype(int)

50/50 [==============================] - 0s 630us/step


In [36]:
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred_binary)
print("Precision:", precision)

Precision: 0.7802746566791511


In [37]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred_binary))

              precision    recall  f1-score   support

         0.0       0.78      0.78      0.78       787
         1.0       0.78      0.78      0.78       798

    accuracy                           0.78      1585
   macro avg       0.78      0.78      0.78      1585
weighted avg       0.78      0.78      0.78      1585



In [18]:
# Export our model to HDF5 file
# nn.save('h5_files/AlphabetSoup_Model4.h5')